In [2]:
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.base import BaseEstimator, TransformerMixin
import numpy as np
import pandas as pd

# Define the Feature Selector
class FeatureSelector(BaseEstimator, TransformerMixin):
    def __init__(self, attribute_names):
        self.attribute_names = attribute_names
        
    def fit(self, X, y=None):
        return self
    
    def transform(self, X):
        return X[self.attribute_names]

# Define the Gap Filler
class CubicInterpolator(BaseEstimator, TransformerMixin):
    def __init__(self, method='cubic', fill_value='extrapolate'):
        self.method = method
        self.fill_value = fill_value
        
    def fit(self, X, y=None):
        return self
    
    def transform(self, X):
        X = X.apply(lambda series: series.interpolate(method=self.method, fill_value=self.fill_value))
        X.fillna(method='ffill', inplace=True)
        X.fillna(method='bfill', inplace=True)
        return X

# Define the Derivative Calculator
class DerivativeCalculator(BaseEstimator, TransformerMixin):
    def __init__(self, dt=0.02):  # Sampling rate of 50 Hz, dt = 0.02 seconds
        self.dt = dt
        
    def fit(self, X, y=None):
        return self
    
    def transform(self, X):
        return X.diff().divide(self.dt)

# Names of the "Z" columns in the raw data frame
selected_names = ['wrist_left_z', 'wrist_right_z']  # You need to adjust this according to your DataFrame's structure

# Create pipeline elements
feature_selector = FeatureSelector(attribute_names=selected_names)
gap_filler = CubicInterpolator()
derivative_calculator = DerivativeCalculator()

# Create the pipeline
pipe1 = Pipeline([
    ('select_features', feature_selector),
    ('fill_gaps', gap_filler),
    ('compute_derivative', derivative_calculator)
])

# Now `pipe1` can be used to process your DataFrame like:
# processed_data = pipe1.fit_transform(raw_data_frame)
